# Mapping from SSH-LCSH to Getty AAT
This notebook demonstrates how we used Wikidata information to derive mappings from "Triples" vocabulary (SSH-LCSH) concepts to Getty Art & Architecture Thesaurus (AAT) concepts. The following diagram illustrates 4 ways that mappings are generated based on the presence of links in Wikidata records referencing both LCSH and AAT resources.

<img src="generating-mappings.png" style="width: 70%; height: 70%">

The SSH-LCSH vocabulary is based on a subset of the Library of Congress Subject Headings (LCSH). There is a 1:1 mapping between SSH-LCSH concepts and LCSH concepts. Wikidata contains certain records that reference both LCSH and AAT concepts. We extract this information and use it to generate direct mappings between SSH-LCSH and AAT concepts. 

Before proceeding, first install (specific versions of) prerequisite Python library dependencies:

In [1]:
%%capture
# load required dependencies
%pip install --upgrade pip
%pip install -r "./requirements.txt"

In [2]:
"""
=============================================================================
Module    : get-SSH-LCSH-to-AAT-mappings.ipynb
Classes   : 
Project   : ATRIUM
Creator   : Ceri Binding, University of South Wales / Prifysgol de Cymru
Contact   : ceri.binding@southwales.ac.uk
Summary   : Python notebook using Wikidata to generate semantic mappings 
            from "Triple" vocabulary (SSH-LCSH) 
            to Getty Art & Architecture Thesaurus (AAT)
            SSH-LCSH: https://www.semantics.gr/authorities/vocabularies/SSH-LCSH 
            AAT: https://vocab.getty.edu/aat
Imports   : os, requests, pyoxigraph, pathlib, rdflib, SPARQLWrapper, datetime
License   : https://github.com/cbinding/ATRIUM-data/blob/main/LICENSE.txt
=============================================================================
History
18/02/2024 CFB Initially created script
17/01/2025 CFB Improved SPARQL queries
=============================================================================
"""
# import required library modules
import os
import requests
from pyoxigraph import *
from pathlib import Path
from rdflib import ConjunctiveGraph, URIRef, Literal
from SPARQLWrapper import SPARQLWrapper, RDFXML
from datetime import datetime as DT


# function to run a SPARQL query against specified external endpoint
# returnFormat supports "xml", "n3", "turtle", "nt" (default), "pretty-xml", "trix", "trig", "nquads", "json-ld" and "hext"
def querySparqlEndpoint(endpointURI: str="", sparqlQuery: str="", returnFormat: str="nt"):
  sparql = SPARQLWrapper(endpointURI)
  sparql.setQuery(sparqlQuery)
  sparql.setMethod("POST")
  sparql.setReturnFormat(RDFXML) # we return RDFXML then convert it to required return format below
  results = sparql.queryAndConvert()
  return results.serialize(format=returnFormat) 


# general function returning timestamp as a string (no allowance for timezone)
def timestamp():
  return DT.now().strftime('%Y-%m-%dT%H:%M:%SZ')  

### Cache the Triple vocabulary (SSH-LCSH)
The [TRIPLE Vocabulary ](https://doi.org/10.12681/semantics.gr/SSH-LCSH) (SSH-LCSH) is a multilingual vocabulary based on a subset of Library of Congress Subject Headings (LCSH). The site reports the SSH-LCSH vocabulary currently (at 16/01/2025) contains 3375 semantic resources (concepts). Download the vocabulary data (here using NTriples RDF format) and save to a local cache file. This only needs to run once, but may be re-run to refresh the local cache file to pick up any changes in the SSH-LCSH data in future.

In [3]:
url = "https://www.semantics.gr/authorities/vocabularies/SSH-LCSH/n-triples"
response = requests.get(url, timeout=30)
with open("./data/SSH-LCSH.nt", "w") as file:
    file.write(response.text)

print(f"[last run {timestamp()}]")

[last run 2025-01-17T11:06:55Z]


### Cache Getty AAT URIs and associated preferred labels
Query the [Getty vocabularies SPARQL endpoint](https://vocab.getty.edu/sparql) to retrieve Art &amp; Architecture Thesaurus (AAT) concept URIs with associated preferred labels (English), and save to a cached NTRiples format RDF file. The labels are only used for review of the mappings produced later (Note result count reported here is min 2 triples per concept). This only needs to run once, but may be re-run to pick up any changes in the AAT data in future.

In [4]:
# Getty AAT SPARQL query to get prefLabels (English) for each Concept (saved as NTriples output)
query = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
CONSTRUCT { 
  ?uri skos:prefLabel ?lbl . 
  ?uri a gvp:Concept .
}
WHERE {
 ?uri a gvp:Concept; skos:inScheme aat:; gvp:prefLabelGVP [gvp:term ?lbl] .
 FILTER(langMatches(lang(?lbl), "en"))
}
"""
results = querySparqlEndpoint(
  endpointURI="https://vocab.getty.edu/sparql", 
  sparqlQuery=query, 
  returnFormat="nt"
)
with open("./data/AAT-prefLabels.nt", "w") as file: 
  file.write(results)
  
counter = len(results.split("\n"))
print(f"{counter} results [last run {timestamp()}]")

116753 results [last run 2025-01-17T11:07:08Z]


### Identify close mappings between LCSH and AAT
Query the Wikidata SPARQL endpoint (https://query.wikidata.org/) to retrieve &lt;LCSH URI&gt; skos:closeMatch &lt;AAT URI&gt; mappings and save them to an NTriples format RDF file. The mappings are generated from Wikidata records referencing both LCSH and AAT identifier properties. Note: Wikidata LCSH URIs retrieved using the wdtn:P244 property may be prefixed 'http://id.loc.gov/authorities/names/' or 'http://id.loc.gov/authorities/' or 'http://id.loc.gov/authorities/subjects/' - to ensure a match to the LCSH URIs as referenced in SSH-LCSH vocabulary, we are concatenating the latter prefix to the LCSH concept identifier, rather than relying on the URI as retrieved from Wikidata.

In [5]:
# Wikidata SPARQL query to generate LCSH-closeMatch-AAT mappings (saved as NTriples output)
query = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
CONSTRUCT { ?lcsh_uri skos:closeMatch ?aat_uri }
WHERE { 
  ?wd_uri wdt:P244 ?lcsh_id; wdtn:P1014 ?aat_uri .
  BIND(IRI(CONCAT('http://id.loc.gov/authorities/subjects/', ?lcsh_id)) AS ?lcsh_uri) .
}
"""

# run the query against the Wikidata SPARQL endpoint
results = querySparqlEndpoint(
  endpointURI="https://query.wikidata.org/sparql", 
  sparqlQuery=query, 
  returnFormat="nt"
)

# write the results to NTriples RDF file
with open("./data/LCSH-closeMatch-AAT.nt", "w") as file: 
  file.write(results)

# Display the number of results retrieved
counter = len(results.split("\n"))
print(f"{counter} results [last run {timestamp()}]")

8048 results [last run 2025-01-17T11:07:14Z]


### Identify broad mappings between LCSH and AAT
Query the Wikidata SPARQL endpoint to retrieve &lt;LCSH&gt; skos:broadMatch &lt;AAT URI&gt; mappings and save them to an NTriples format RDF file. These mappings are generated from Wikidata records referencing LCSH but not AAT, and being a subclass of _another_ Wikidata record having an AAT identifier property. Note: Same caveat as above regarding prefixing the LCSH identifiers.

In [7]:
# Wikidata SPARQL query to generate LCSH-broadMatch-AAT mappings (saved as NTriples output)
query = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
CONSTRUCT { ?lcsh_uri skos:broadMatch ?aat_uri }
WHERE { 
  ?wd_uri wdt:P244 ?lcsh_id ; wdt:P279 [ wdtn:P1014 ?aat_uri ] .
  MINUS { ?wd_uri wdtn:P1014 ?x }   
  BIND(IRI(CONCAT('http://id.loc.gov/authorities/subjects/', ?lcsh_id)) AS ?lcsh_uri) .
}
"""

# run the query against the Wikidata SPARQL endpoint
results = querySparqlEndpoint(
  endpointURI="https://query.wikidata.org/sparql", 
  sparqlQuery=query, 
  returnFormat="nt"
)

# write the results to NTriples RDF file
with open("./data/LCSH-broadMatch-AAT.nt", "w") as file: 
  file.write(results)

# Display the number of results retrieved
counter = len(results.split("\n"))
print(f"{counter} results [last run {timestamp()}]")

10028 results [last run 2025-01-17T11:08:28Z]


### Combine all to an RDF graph instance for subsequent querying
Combine the SSH-LCSH vocabulary, the AAT data and the LCSH -&gt; AAT mappings (NTriples format RDF files) to an rdflib ConjunctiveGraph instance. 

In [51]:
graph = ConjunctiveGraph(store="Oxigraph") # speed advantage over plain rdflib.Graph
# persisting to disk. Not strictly necessary here though
# if not os.path.isdir(os.path.join(os.getcwd(), "graph_dir")): graph.open("graph_dir")
# import RDF files
graph.parse("./data/SSH-LCSH.nt")
graph.parse("./data/AAT-prefLabels.nt")
graph.parse("./data/LCSH-closeMatch-AAT.nt")
graph.parse("./data/LCSH-broadMatch-AAT.nt")

<Graph identifier=Na3c57fefd9a3412ead2341ee7473674b (<class 'rdflib.graph.Graph'>)>

## Fix target LCSH URIs before proceeding 
SSH-LCSH (Triples vocab) data may occasionally specify skos:exactMatch to a deprecated LCSH concept URI:
e.g. <http://semantics.gr/authorities/SSH-LCSH/sh98005996> skos:exactMatch <http://id.loc.gov/authorities/subjects/sh2014001211> .
the LCSH target SHOULD be <http://id.loc.gov/authorities/subjects/sh98005996> - so construct additional triples 
to ensure we have <SSH-LCSH> skos:exactMatch <LCSH> relationship - identifying the correct target URI
use LCSH URI prefix and SSH-LCSH identifier to produce (e.g.) "http://id.loc.gov/authorities/subjects/sh98005996"

In [53]:
# 
query = """
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX gvp: <http://vocab.getty.edu/ontology#>
  CONSTRUCT { ?ssh_uri skos:exactMatch ?lcsh_uri }
  WHERE {    
    ?ssh_uri a skos:Concept .   
    BIND(STRAFTER(STR(?ssh_uri), "http://semantics.gr/authorities/SSH-LCSH/") AS ?id) .
    BIND(IRI(CONCAT("http://id.loc.gov/authorities/subjects/", ?id)) AS ?lcsh_uri) .
}
"""
results = graph.query(query)

# save the resultant constructed triples to an NTriples RDF format text file
results.serialize(destination="./data/SSH-LCSH-exactMatch-LCSH.nt", format="nt")

# add the resultant constructed triples back into the existing graph
graph.parse("./data/SSH-LCSH-exactMatch-LCSH.nt")

# function to check how many SSH-LCSH concepts remain unmapped
def countUnmappedConcepts():
    query = """
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX gvp: <http://vocab.getty.edu/ontology#>
        SELECT (COUNT(DISTINCT ?ssh_uri) AS ?counter) WHERE {
        ?ssh_uri a skos:Concept .
        MINUS { ?ssh_uri ?property [a gvp:Concept] }
        }
    """
    output = graph.query(query)
    # we only want the count (maybe a better way to do this?)
    for result in output:
        return result[0]
        break  

# should be 3375 results (1 per SSH-LCSH concept)
print(f"{len(results)} results [last run {timestamp()}]")

3375 results [last run 2025-01-17T12:29:48Z]


### Generate SSH-LCSH to AAT mappings
Use the LCSH-closeMatch-AAT mappings to produce &lt;SSH-LCSH&gt; skos:closeMatch &lt;AAT&gt; mappings. Save results to an NTriples RDF file, then import the newly produced mappings back into the graph triple store.

In [54]:
# create SSH-LCSH closeMatch AAT relationships based on Wikidata LCSH to AAT relationships
query = """
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX gvp: <http://vocab.getty.edu/ontology#>
  CONSTRUCT { ?ssh_uri skos:closeMatch ?aat_uri }
  WHERE {
    ?ssh_uri a skos:Concept; skos:exactMatch ?lcsh_uri .
    ?lcsh_uri skos:closeMatch ?aat_uri .
    ?aat_uri a gvp:Concept .
    MINUS { ?ssh_uri ?property [a gvp:Concept] }
}
"""
results = graph.query(query)

# save the resultant constructed mappings to an NTriples RDF format text file
results.serialize(destination="./data/SSH-LCSH-closeMatch-AAT.nt", format="nt")

# add these mappings back into the existing graph
graph.parse("./data/SSH-LCSH-closeMatch-AAT.nt")

# display number of results and number of SSH-LCSH concepts that remain unmapped
unmapped = countUnmappedConcepts()
print(f"{len(results)} results. {unmapped} unmapped concepts [last run {timestamp()}]")

366 results. 2980 unmapped concepts [last run 2025-01-17T12:29:59Z]


For remaining SSH-LCSH concepts not yet mapped, create &lt;SSH-LCSH&gt; skos:broadMatch &lt;AAT&gt; relationships where possible

In [55]:
# create SSH-LCSH broadMatch AAT relationships based on Wikidata LCSH to AAT relationships
query = """
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX gvp: <http://vocab.getty.edu/ontology#>
  CONSTRUCT { ?ssh_uri skos:broadMatch ?aat_uri }
  WHERE {
    ?ssh_uri a skos:Concept; skos:exactMatch ?lcsh_uri .
    ?lcsh_uri skos:broadMatch ?aat_uri .
    ?aat_uri a gvp:Concept .
    MINUS { ?ssh_uri ?property [a gvp:Concept] }
}
"""
results = graph.query(query)

# save the resultant constructed mappings to an NTriples RDF format text file
results.serialize(destination="./data/SSH-LCSH-broadMatch-AAT-A.nt", format="nt")

# add these mappings back into the existing graph
graph.parse("./data/SSH-LCSH-broadMatch-AAT-A.nt")

# display number of results and number of SSH-LCSH concepts that remain unmapped
unmapped = countUnmappedConcepts()
print(f"{len(results)} results. {unmapped} unmapped concepts [last run {timestamp()}]")

431 results. 2639 unmapped concepts [last run 2025-01-17T12:30:05Z]


For remaining SSH-LCSH concepts not yet mapped, if the parent SSH-LCSH concept has a skos:closeMatch mapping to an AAT concept, create &lt;SSH-LCSH&gt; skos:broadMatch &lt;AAT&gt; mapping 

In [56]:
# create skos:broadMatch relationships for concepts
# not yet mapped, using vocabulary broader relationship
query = """
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX gvp: <http://vocab.getty.edu/ontology#>
  CONSTRUCT { ?ssh_uri skos:broadMatch ?aat_uri }
  WHERE {
    ?ssh_uri a skos:Concept; skos:broader ?broader_uri .
    ?broader_uri a skos:Concept; skos:closeMatch ?aat_uri .
    ?aat_uri a gvp:Concept .
    MINUS { ?ssh_uri ?property [a gvp:Concept] }
  }
"""
results = graph.query(query)

# save the resultant constructed mappings to an NTriples RDF format text file
results.serialize(destination="./data/SSH-LCSH-broadMatch-AAT-B.nt", format="nt")

# add the resultant mappings back into the existing graph
graph.parse("./data/SSH-LCSH-broadMatch-AAT-B.nt")

# display number of results and number of SSH-LCSH concepts that remain unmapped
unmapped = countUnmappedConcepts()
print(f"{len(results)} results. {unmapped} unmapped concepts [last run {timestamp()}]")

2399 results. 592 unmapped concepts [last run 2025-01-17T12:30:14Z]


For remaining SSH-LCSH concepts not yet mapped, if the parent SSH-LCSH concept has a skos:broadMatch mapping to an AAT concept, create &lt;SSH-LCSH&gt; skos:relatedMatch &lt;AAT&gt; mapping 

In [57]:
# create skos:relatedMatch relationships for concepts
# not yet mapped, using vocabulary broader relationship
query = """
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX gvp: <http://vocab.getty.edu/ontology#>
  CONSTRUCT { ?ssh_uri skos:relatedMatch ?aat_uri }
  WHERE {
    ?ssh_uri a skos:Concept; skos:broader ?broader_uri .
    ?broader_uri a skos:Concept; skos:broadMatch ?aat_uri .
    ?aat_uri a gvp:Concept .
    MINUS { ?ssh_uri ?property [a gvp:Concept] }
  }
"""
results = graph.query(query)

# save the resultant constructed mappings to an NTriples RDF format text file
results.serialize(destination="./data/SSH-LCSH-relatedMatch-AAT.nt", format="nt")

# add the resultant mappings back into the existing graph
graph.parse("./data/SSH-LCSH-relatedMatch-AAT.nt")

# display number of results and number of SSH-LCSH concepts that remain unmapped
unmapped = countUnmappedConcepts()
print(f"{len(results)} results. {unmapped} unmapped concepts [last run {timestamp()}]")

344 results. 310 unmapped concepts [last run 2025-01-17T12:30:24Z]


### Create output for review
Create a CSV listing of the mappings created, for manual review

In [58]:
# list the mappings created (for review)
query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX gvp: <http://vocab.getty.edu/ontology#>
    SELECT DISTINCT ?ssh_uri (str(?ssh_lbl) AS ?ssh_label) ?rel ?aat_uri (str(?aat_lbl) AS ?aat_label)    
    WHERE {
        ?ssh_uri skos:closeMatch|skos:broadMatch|skos:relatedMatch ?aat_uri . 
        ?ssh_uri a skos:Concept .
        ?aat_uri a gvp:Concept .         
        ?ssh_uri ?rel ?aat_uri .        
        OPTIONAL { 
            ?ssh_uri skos:prefLabel ?ssh_lbl . 
            FILTER(langMatches(lang(?ssh_lbl), "en")) 
        }
        OPTIONAL { 
            ?aat_uri skos:prefLabel ?aat_lbl . 
            FILTER(langMatches(lang(?aat_lbl), "en")) 
        }
    }
"""
mappings = graph.query(query)
mappings.serialize(destination="./data/SSH-LCSH-matched-AAT.csv", format="csv")

print(f"{len(mappings)} mappings [last run {timestamp()}]")

3589 mappings [last run 2025-01-17T12:30:46Z]


Display a subset of the mappings produced, for review. Note any SSH-LCSH records showing a blank label are concepts having no English preferred label present in the vocabulary, the mapping still works regardless of this

In [59]:
# display a subset of the mappings produced for review. Records with no SSH-LCSH label displayed indicates 
# there is no English preferred label for the concept in SSH-LCSH, but this does not affect the mapping
import pandas as pd
from IPython.display import display, HTML
df = pd.DataFrame(mappings)
display(HTML(df.fillna("").to_html(index=False, header=False, max_rows=30)))

http://semantics.gr/authorities/SSH-LCSH/sh85028889,,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300055912,commedia dell'arte
http://semantics.gr/authorities/SSH-LCSH/sh85081863,Mass media,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300055812,mass media
http://semantics.gr/authorities/SSH-LCSH/sh85066150,Information science,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054574,information science
http://semantics.gr/authorities/SSH-LCSH/sh85076723,Library science,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054576,library science
http://semantics.gr/authorities/SSH-LCSH/sh85108411,Psychoanalysis,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054450,psychoanalysis
http://semantics.gr/authorities/SSH-LCSH/sh85148092,Word (Linguistics),http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300250895,words
http://semantics.gr/authorities/SSH-LCSH/sh85036316,Decorative arts,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054168,decorative arts
http://semantics.gr/authorities/SSH-LCSH/sh85119950,Semiotics,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054254,semiotics
http://semantics.gr/authorities/SSH-LCSH/sh85015738,Books,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300028051,books
http://semantics.gr/authorities/SSH-LCSH/sh85006611,Architecture,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300263552,architecture
http://semantics.gr/authorities/SSH-LCSH/sh89004185,Puppet theater,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300248147,puppetry


Produce a CSV listing of the remaining SSH-LCSH concepts not yet mapped, for manual review

In [60]:
# list the remaining records not mapped
query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX gvp: <http://vocab.getty.edu/ontology#>
    SELECT DISTINCT ?ssh_uri (str(?lbl) AS ?label) WHERE {
    ?ssh_uri a skos:Concept .
    OPTIONAL { 
        ?ssh_uri skos:prefLabel ?lbl . 
        FILTER(langMatches(lang(?lbl), "en")) 
    }
    MINUS { ?ssh_uri ?property [a gvp:Concept] }
    }
"""
unmapped = graph.query(query)
unmapped.serialize(destination="./data/SSH-LCSH-nomatch-AAT.csv", format="csv")

print(f"{len(unmapped)} unmapped [last run {timestamp()}]")

310 unmapped [last run 2025-01-17T12:30:55Z]


Display a subset of the unmapped SSH-LCSH concepts here, for review.

In [61]:
# display the unmapped records here for review
import pandas as pd
from IPython.display import display, HTML
df = pd.DataFrame(unmapped)
display(HTML(df.fillna("").to_html(index=False, header=False, max_rows=25)))

http://semantics.gr/authorities/SSH-LCSH/sh85116920,Salvadoran literature
http://semantics.gr/authorities/SSH-LCSH/sh85072864,Komi-Permyak literature
http://semantics.gr/authorities/SSH-LCSH/sh85057146,"Greek essays, Modern"
http://semantics.gr/authorities/SSH-LCSH/sh85054380,German literature
http://semantics.gr/authorities/SSH-LCSH/sh88005294,"Rwanda, Literatures"
http://semantics.gr/authorities/SSH-LCSH/sh85119451,Sects
http://semantics.gr/authorities/SSH-LCSH/sh85073564,
http://semantics.gr/authorities/SSH-LCSH/sh85126289,Spanish poetry
http://semantics.gr/authorities/SSH-LCSH/sh2012000722,"Moldova, Literatures"
http://semantics.gr/authorities/SSH-LCSH/sh85025095,Christian life
http://semantics.gr/authorities/SSH-LCSH/sh2002005701,Wild Man of Orford (Legendary character)


Create a listing of all generated mappings in [TriG](https://www.w3.org/TR/trig/) RDF format

In [62]:
from rdflib.namespace import XSD

# remove the named graph of mappings if it already exists
graphURI = "http://mappings-SSH-LCSH-to-AAT/"
mappings = graph.get_context(URIRef(graphURI))
graph.remove_context(mappings)

# get the generated mappings as RDF NTriples
query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX gvp: <http://vocab.getty.edu/ontology#>
    CONSTRUCT { ?ssh_uri ?rel ?aat_uri }
    WHERE {  
    ?ssh_uri skos:closeMatch|skos:broadMatch|skos:relatedMatch ?aat_uri .
    ?ssh_uri a skos:Concept . 
    ?aat_uri a gvp:Concept .
    ?ssh_uri ?rel ?aat_uri . 
    } 
"""
results = graph.query(query)
# add named graph URI to each result for subsequent quads output
results = list(map(lambda t: t + (graphURI,), results))
# add the mappings (now quads) to the triple store
graph.addN(results)

# add some metadata to the named graph of mappings
graph.addN([
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/identifier"),
        Literal(graphURI),
        URIRef(graphURI),
    ),    
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/date"),
        Literal(timestamp(), datatype=XSD.date),
        URIRef(graphURI),
    ),
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/source"),
        Literal("https://query.wikidata.org/"),
        URIRef(graphURI),
    ),    
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/title"),
        Literal("SSH-LCSH-to-AAT mappings"),
        URIRef(graphURI),
    ),
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/description"),
        Literal("SSH-LCSH-to-AAT mappings generated from Wikidata"),
        URIRef(graphURI),
    ),
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/format"),
        Literal("application/trig"),
        URIRef(graphURI),
    )
])


# export the entire named graph of mappings to TRIG format RDF file
mappings = graph.get_context(URIRef(graphURI))
mappings.serialize(destination="./data/SSH-LCSH-matched-AAT.trig", format="trig")

print(f"{len(results)} results [last run {timestamp()}]")

3588 results [last run 2025-01-17T12:31:10Z]
